# Парсинг данных по статьям через API eLibrary для проекта "Паттерны коллаборации в Российском социологическом сообществе"

В этом файле производится выгрузка данных eLibrary для проекта Лаборатории по гранту РНФ "Паттерны коллаборации в российском социологическом сообществе: структура научных школ и возможные точки роста". Осуществляется импорт необходимых библиотек, загружаются id необходимых к выгрузке статьи, производится сама выгрузка основного массива данных.

Основной код выгрузки был подготовлен Львом Ципесом, в доработке кода и выгрузке данных участвовали Василиса Ващенко и Лика Капустина. Выгрузка производилась в июне-июле 2022 года. Эта версия была закончена и прокомментирована Ликой Капустиной.

Код, представленный здесь, можно использовать для выгрузки данных о статьях через API Elibrary. Предварительно вам нужно заключить договор с Elibrary, и тогда вы получите доступ к одному из сервисов API по IP-адресу, указанному в договоре. 

**Что вам нужно:**
* Файл формата `.txt` с id статей, информацию о которых нужно выгрузить через API Elibrary.

**Процесс выгрузки данных о статьях через API Elibrary происходит в несколько шагов:**

* **Шаг 1: подготовка к работе;**
* **Шаг 2: выгрузка данных**
* **Шаг 3: сохранение данных**

**В результате вы получите файлы:**
* Промежуточные файлы с данными в формате `.csv` и `.xlsx`, которые сохраняются локально в рабочую директорию каждую 1000 итераций (с названием в формате `df_текущая дата_число строк_rows.формат файла`);
* `final_data.csv` - `pandas.DataFrame` с информацией о всех статьях, находящихся в изначальном списке, в формате `.csv`;
* `final_data.xlsx` - `pandas.DataFrame` с информацией о всех статьях, находящихся в изначальном списке, в формате `.xlsx`;

<h2><center> Шаг 1. Подготовка к работе </center></h2>

### Загрузка необходимых библиотек

In [2]:
#pip install xmltodict
from numpy import fromstring # приведение данных к строке
import requests # модуль для запросов к данным по ссылкам
import xmltodict # работа с xml
import pandas as pd # модуль для работы с таблицами
from tqdm import tqdm # модуль для работы с tqdm
from time import sleep # импортируем функцию сна 
import warnings # модуль с предупреждениями
warnings.filterwarnings("ignore") # выбираем игнорировать предупреждения
import xml.etree.ElementTree as ET # работа с xml
from xml.etree.ElementTree import parse # работа с xml
from urllib.request import urlopen # работа с запросами
import pickle # формат данных pickle
import datetime # работа с данными

### Проверка ip-адреса

Выгрузка данных может производиться только по ip-адресу, указанному в договоре с Elibrary. С помощью кода ниже вы можете проверить, с какого ip-адреса вы подключились. Важно перед началом работы отключить VPN, если он у вас включен.

In [3]:
import http.client
conn = http.client.HTTPConnection("ifconfig.me")
conn.request("GET", "/ip")
conn.getresponse().read()

b'92.242.58.41'

На этом этапе загружаем текстовый файл с id публикаций – вы получите их от Elibrary после заключения договора. В нашем случае в файле `421_1.txt` лежали id публикаций социологических статей. Мы отталкивались от этого списка – и получали данные от API Elibrary по id этих статей.

In [5]:
with open('421_1.txt') as f: # замените 421_1.txt на свой адрес файла с id статей
    lines = f.readlines()

На следующем этапе мы проверяем число объектов в файле и сравниваем их с числом статей, которое обговаривалось на этапе общения с представителями Elibrary. В нашем случае предполагалось, что мы получим данные по 66 473 статьям. Если ниже вы ввели число статей, которое должны получить и при сравнении с актуальным числом статей в `.txt` файле получили `True` - все ок.

In [6]:
number_of_needed_articles = int(input('Введите, информацию по какому числу статей вы должны получить: '))
a = len(lines)
print(a == 66473)

True


<h2><center>Шаг 2. Выгрузка данных</center></h2>

В этой части производится попытка выгрузка информации по нужным нам статьям.

**Обратите внимание:** каждые 1000 строк в `pandas.DataFrame` с данными о статьях, происходит сохранение файлов локально (на ваш компьютер) в двух форматах - `.csv` и `.xlsx`.

Эти файлы сохраняются в вашу рабочую директорию (в папке, где находится файл). Чтобы точно проверить, где находится ваша рабочая директория, используйте код ниже:
```Python
    import os
    os.getcwd()
```

Итак, каждую 1000 итераций ваши файлы сохраняются локально в рабочую директорию. Название файла выглядит так:
`df_дата_число строк_rows.формат`. Файлы занимают место, и если вы не хотите чтобы файлы сохранялись локально, то  просто удалите следующие строчки в конце ячейки: 

```Python
    df.to_csv(f'df_{current_date}_{counter}_rows.csv')
    df.to_excel(f'df_{current_date}_rows.xlsx')
```

**У вас может возникнуть проблема с выгрузкой данных из-за качества Wi-fi соединения; в таком случае стоит:**
* Обновить параметр `counter` до числа выгруженных строк;
* Обновить параметр, который передается внутрь цикла `for`;
* Запустить код заново

Продемонстрирую на примере. Изначально код начинался так: 

```Python
    counter = 0

    for l in tqdm(range(len(lines))):
```

Предположим, что у вас произошла ошибка на 2001-ой итерации цикла (то есть, после обработки 2000 статей). Тогда вам следует поменять первую часть кода на:

```Python
    counter = 2000

    for l in tqdm(range(len(lines[2000:]))):
```

И после этого перезапустить код заново.

In [3]:
# Создаем pandas.DataFrame с заранее заданными колонками
df = pd.DataFrame(columns =    ['id',
                               'genreId',
                               
                               'author_1_id',
                               'author_1_affiliation_id',
                               'author_1_ru last name',
                               'author_1_ru initials',
                               'author_1_ru affiliation',
                               'author_1_eng last name',
                               'author_1_eng initials',
                               'author_1_eng affiliation',
                               
                               'author_2_id',
                               'author_2_affiliation_id',
                               'author_2_ru last name',
                               'author_2_ru initials',
                               'author_2_ru affiliation',
                               'author_2_eng last name',
                               'author_2_eng initials',
                               'author_2_eng affiliation',
                               
                               'author_3_id',
                               'author_3_affiliation_id',
                               'author_3_ru last name',
                               'author_3_ru initials',
                               'author_3_ru affiliation',
                               'author_3_eng last name',
                               'author_3_eng initials',
                               'author_3_eng affiliation',
                               
                               'author_4_id',
                               'author_4_affiliation_id',
                               'author_4_ru last name',
                               'author_4_ru initials',
                               'author_4_ru affiliation',
                               'author_4_eng last name',
                               'author_4_eng initials',
                               'author_4_eng affiliation',
                               
                               'author_5_id',
                               'author_5_affiliation_id',
                               'author_5_ru last name',
                               'author_5_ru initials',
                               'author_5_ru affiliation',
                               'author_5_eng last name',
                               'author_5_eng initials',
                               'author_5_eng affiliation',
                               
                               'author_6_id',
                               'author_6_affiliation_id',
                               'author_6_ru last name',
                               'author_6_ru initials',
                               'author_6_ru affiliation',
                               'author_6_eng last name',
                               'author_6_eng initials',
                               'author_6_eng affiliation',
                               
                               'author_7_id',
                               'author_7_affiliation_id',
                               'author_7_ru last name',
                               'author_7_ru initials',
                               'author_7_ru affiliation',
                               'author_7_eng last name',
                               'author_7_eng initials',
                               'author_7_eng affiliation',
                               
                               'author_8_id',
                               'author_8_affiliation_id',
                               'author_8_ru last name',
                               'author_8_ru initials',
                               'author_8_ru affiliation',
                               'author_8_eng last name',
                               'author_8_eng initials',
                               'author_8_eng affiliation',
                               
                               'title_ru',
                               'title_eng',
                               'year',
                               'keywords_ru',
                               'keywords_eng',
                               'abstract_ru',
                               'abstract_eng',
                               'rubric',
                               'rubric_code',
                              ])

In [89]:
counter = 0 #счетчик
current_date = str(datetime.datetime.today().date()) # сегодняшняя дата


for l in tqdm(range(len(lines))): 
    line = last_part[l]
    code = line
    url = "http://elibrary.ru/projects/API-NEB/API_NEB.aspx?ucode=6b143b66-c479-4d49-91ad-44815b42f0d2&sid=026&itemid="+code
    response = requests.get(url)
    data = xmltodict.parse(response.content)
    tree_response = urlopen(url).read()
    tree_data = ET.fromstring(tree_response)
    #print(ET.tostring(tree_data, encoding='utf8').decode('utf8')) #- вывести XML файл
    
    
    rus_auth_names = list()
    rus_auth_initials = list()
    rus_auth_affiliations = list()
    eng_auth_names = list()
    eng_auth_initials = list()
    eng_auth_affiliations = list()
    
    eng_auth_id=list()
    eng_afil_id=list()

    rus_auth_id=list()
    rus_afil_id=list()

    final_ids_ordered = list()
    final_auths_ordered = list()

    #### блок работы с авторами. 

    #xxx_auth_names - хранит в себе список фамилий авторов на языке xxx. 
    #xxx_auth_initials - хранит в себе список инициалов авторов на языке xxx. 
    #ххх_auth_affiliations - хранит в себе список списков аффилиаций авторов на языке xxx.
    #далее в df каждая ячейка тематики yyy будет выглядить так:
    #    [rus_auth_yyy],[eng_auth_yyy]
    try:
        author_data_for_numerizer = data['root']['item']['authors']['author']
        if type(author_data_for_numerizer) != list: #если всего один автор, то в XML он хранится в виде словаря а не списка словарей. поэтому, чтобы работало, делаем список словарей в любом случае
            author_data_for_numerizer = [author_data_for_numerizer]

        for j in range(1,int(max([x['@num'] for x in author_data_for_numerizer]))+1):


            author_data = list(filter(lambda z: z['@num'] == str(j), author_data_for_numerizer))

            rus_auth_names = rus_auth_names+[x['lastname'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', author_data))]
            try:
                eng_auth_names = eng_auth_names+[x['lastname'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', author_data))]
            except:
                pass


            rus_auth_initials = rus_auth_initials+[x['initials'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', author_data))]
            try:
                eng_auth_initials = eng_auth_initials+[x['initials'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', author_data))]
            except:
                pass

            try:
                rus_auth_affiliations = rus_auth_affiliations+[k['#text'] for k in [l['affiliation'] for l in [x['affiliations'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU',  author_data))]]]
                try:
                    eng_auth_affiliations = eng_auth_affiliations+[k['#text'] for k in [l['affiliation'] for l in [x['affiliations'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN',  author_data))]]]
                except:
                    pass
            except:
                pass
            
        #работа с id
            for kid in tree_data.iter('authors'):
                #print(tostring(kid, encoding='utf8').decode('utf8')) 
                for child in kid.iter('author'):


                    #print(tostring(child, encoding='utf8').decode('utf8'))

                    if child.attrib['lang'] == 'EN' and child.attrib['num'] == str(j):
                        eng_auth_id.append(child.attrib['id'])

                        if 'affiliation' in [elem.tag for elem in child.iter()]:

                            for daughter in child.iter('affiliation'):
                                eng_afil_id.append(daughter.attrib['id'])
                        else:
                            eng_afil_id.append('none')

                    if child.attrib['lang'] == 'RU' and child.attrib['num'] == str(j):
                        rus_auth_id.append(child.attrib['id'])

                        if 'affiliation' in [elem.tag for elem in child.iter()]:

                            for daughter in child.iter('affiliation'):
                                rus_afil_id.append(daughter.attrib['id'])
                        else:
                            rus_afil_id.append('none')

        while len(rus_afil_id)<9:
            rus_afil_id.append('none')
        while len(eng_afil_id)<9:
            eng_afil_id.append('none')

        while len(rus_auth_id)<9:
            rus_auth_id.append('none')
        while len(eng_auth_id)<9:
            eng_auth_id.append('none')

        for i in range(len(rus_afil_id)):
            if rus_afil_id[i]=='none':
                if eng_afil_id[i]!='none':
                    final_ids_ordered.append(eng_afil_id[i])
                else:
                    final_ids_ordered.append(rus_afil_id[i])
            else:
                final_ids_ordered.append(rus_afil_id[i])        

        for i in range(len(rus_auth_id)):
            if rus_auth_id[i]=='none':
                if eng_auth_id[i]!='none':
                    final_auths_ordered.append(eng_auth_id[i])
                else:
                    final_auths_ordered.append(rus_auth_id[i])
            else:
                final_auths_ordered.append(rus_auth_id[i])

        
        while len(rus_auth_names)<9:
            rus_auth_names.append('none')
            rus_auth_initials.append('none')
        while len(eng_auth_names)<9:
            eng_auth_names.append('none')
            eng_auth_initials.append('none')
        while len(rus_auth_affiliations)<9:
            rus_auth_affiliations.append('none')
        while len(eng_auth_affiliations)<9:
            eng_auth_affiliations.append('none')
        #while len(author_ids)<9:
        #    author_ids.append('none')
            
    except:
        final_ids_ordered = ['none' for x in range(9)]
        final_auths_ordered = ['none' for x in range(9)]
        rus_auth_names = ['none' for x in range(9)]
        rus_auth_initials = ['none' for x in range(9)]
        rus_auth_affiliations = ['none' for x in range(9)]
        eng_auth_names = ['none' for x in range(9)]
        eng_auth_initials = ['none' for x in range(9)]
        eng_auth_affiliations = ['none' for x in range(9)]


    
    
    #### блок работы с названиями работ.
    
    
    try:
        titles = data['root']['item']['titles']['title']
        if type(titles) != list:
            titles = [titles]             
        ru_title = [x['#text'] for x in list(filter(lambda alpha: alpha['@lang'] == 'RU', titles))][0]
        try:
            eng_title = [x['#text'] for x in list(filter(lambda alpha: alpha['@lang'] == 'EN', titles))][0]
        except:
            eng_title = 'none'
    except:
        ru_title = data['root']['item']['title']
        eng_title = 'none'
    
    
    #### блок работы с абстрактами
  
    
    try: #делаю траем, тк абстракта в XML может не быть
        abstract_data = data['root']['item']['abstracts']['abstract']

        
        ru_abstract = [x['#text'] for x in list(filter(lambda gamma: gamma['@lang'] == 'RU',  abstract_data))][0]
        try:
            eng_abstract = [x['#text'] for x in list(filter(lambda gamma: gamma['@lang'] == 'EN',  abstract_data))][0]
        except:
            eng_abstract = 'none'
    except:
        ru_abstract = 'none' 
        eng_abstract = 'none'
    
    
    #### блок работы с ключевыми словами

    
    try: #делаю траем, тк ключевых слов в XML может не быть
        keywords_data = data['root']['item']['keywords']['keyword']
        ru_keywords = [x['#text'] for x in list(filter(lambda delta: delta['@lang'] == 'RU', keywords_data))]
        try:
            eng_keywords = [x['#text'] for x in list(filter(lambda delta: delta['@lang'] == 'EN', keywords_data))]
        except:
            eng_keywords = 'none'
    except:
        ru_keywords = 'none'
        eng_keywords = 'none'
    if eng_keywords == []:
        eng_keywords = 'none'

    ### блок работы с кодами DOI и EDN
    code_data = data['root']['item']['itemCodes']['code']
    try:
        code_DOI = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'DOI', code_data))]
        try:
            code_EDN = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'EDN', code_data))]
            try: 
                code_UDK = [x['#text'] for x in list(filter(lambda delta: delta['@type'] == 'УДК', code_data))]
            except:
                code_UDK = 'none'
        except:
            code_EDN = 'none'
    except:
        code_DOI = 'none'
        code_EDN = 'none'
        code_UDK = 'none'
    if code_DOI == []:
        code_DOI = 'none'
    if code_EDN == []:
        code_EDN = 'none'
    if code_UDK == []:
        code_UDK = 'none'
    
        
    
    ### блок работы с финансированием
    try: #делаю через try, тк информации о финансировании в XML может не быть
        fin_data = data['root']['item']['support']['supported']
        fin = fin_data['#text']#[x['#text'] for x in list(fin_data)]
        #print(fin)
    except:
        fin = None
    if fin == None:
        fin = 'none'
        
    ### блок работы с публикацией в журнале 
    try:
        number = data['root']['item']['number']
    except:
        number = 'none'
    try:
        jp = data['root']['item']['pages']
    except: 
        jp = 'none'
    try:
        np = data['root']['item']['pagesNumber']
    except:
        np = 'none'
    try: 
        volume = data['root']['item']['volume']
    except:
        volume = 'none'
    try:
        url = data['root']['item']['url']
    except:
        url = 'none'
        
    try:
        pub_data = data['root']['item']['publisher']
        publisher = pub_data['#text']
        publisher_id = publisher_id['@id']
    except:
        publisher = 'none'
        publisher_id = 'none'
    #### блок добавления новой строки
    
    # new_row - создаем pandas.DataFrame с информацией об одной статье, в формате:
    # {'название колонки 1': значение колонки 1, 'название колонки 2': знаачение колонки 2} и т.д.
    new_row = {'id':data['root']['item']['@id'],
               'genreId':data['root']['item']['@genreId'],
             
               'author_1_id':final_auths_ordered[0], 
               'author_1_affiliation_id':final_ids_ordered[0],
               'author_1_ru last name':rus_auth_names[0], 
               'author_1_ru initials':rus_auth_initials[0],
               'author_1_ru affiliation':rus_auth_affiliations[0], 
               'author_1_eng last name':eng_auth_names[0], 
               'author_1_eng initials':eng_auth_initials[0],
               'author_1_eng affiliation':eng_auth_affiliations[0], 

               'author_2_id':final_auths_ordered[1], 
               'author_2_affiliation_id':final_ids_ordered[1],               
               'author_2_ru last name':rus_auth_names[1], 
               'author_2_ru initials':rus_auth_initials[1],
               'author_2_ru affiliation':rus_auth_affiliations[1], 
               'author_2_eng last name':eng_auth_names[1], 
               'author_2_eng initials':eng_auth_initials[1],
               'author_2_eng affiliation':eng_auth_affiliations[1], 

               'author_3_id':final_auths_ordered[2], 
               'author_3_affiliation_id':final_ids_ordered[2],               
               'author_3_ru last name':rus_auth_names[2], 
               'author_3_ru initials':rus_auth_initials[2],
               'author_3_ru affiliation':rus_auth_affiliations[2], 
               'author_3_eng last name':eng_auth_names[2], 
               'author_3_eng initials':eng_auth_initials[2],
               'author_3_eng affiliation':eng_auth_affiliations[2], 

               'author_4_id':final_auths_ordered[3], 
               'author_4_affiliation_id':final_ids_ordered[3],               
               'author_4_ru last name':rus_auth_names[3], 
               'author_4_ru initials':rus_auth_initials[3],
               'author_4_ru affiliation':rus_auth_affiliations[3], 
               'author_4_eng last name':eng_auth_names[3], 
               'author_4_eng initials':eng_auth_initials[3],
               'author_4_eng affiliation':eng_auth_affiliations[3], 

               'author_5_id':final_auths_ordered[4], 
               'author_5_affiliation_id':final_ids_ordered[4],           
               'author_5_ru last name':rus_auth_names[4], 
               'author_5_ru initials':rus_auth_initials[4],
               'author_5_ru affiliation':rus_auth_affiliations[4], 
               'author_5_eng last name':eng_auth_names[4], 
               'author_5_eng initials':eng_auth_initials[4],
               'author_5_eng affiliation':eng_auth_affiliations[4], 

               'author_6_id':final_auths_ordered[5], 
               'author_6_affiliation_id':final_ids_ordered[5],               
               'author_6_ru last name':rus_auth_names[5], 
               'author_6_ru initials':rus_auth_initials[5],
               'author_6_ru affiliation':rus_auth_affiliations[5], 
               'author_6_eng last name':eng_auth_names[5], 
               'author_6_eng initials':eng_auth_initials[5],
               'author_6_eng affiliation':eng_auth_affiliations[5], 

               'author_7_id':final_auths_ordered[6], 
               'author_7_affiliation_id':final_ids_ordered[6],               
               'author_7_ru last name':rus_auth_names[6], 
               'author_7_ru initials':rus_auth_initials[6],
               'author_7_ru affiliation':rus_auth_affiliations[6], 
               'author_7_eng last name':eng_auth_names[6], 
               'author_7_eng initials':eng_auth_initials[6],
               'author_7_eng affiliation':eng_auth_affiliations[6], 

               'author_8_id':final_auths_ordered[7], 
               'author_8_affiliation_id':final_ids_ordered[7],               
               'author_8_ru last name':rus_auth_names[7], 
               'author_8_ru initials':rus_auth_initials[7],
               'author_8_ru affiliation':rus_auth_affiliations[7], 
               'author_8_eng last name':eng_auth_names[7], 
               'author_8_eng initials':eng_auth_initials[7],
               'author_8_eng affiliation':eng_auth_affiliations[7], 
               
               'title_ru':ru_title, 
               'title_eng':eng_title, 
               
               'year':data['root']['item']['year'],
               
               'keywords_ru':ru_keywords,
               'keywords_eng':eng_keywords,
               
               'abstract_ru':ru_abstract,
               'abstract_eng':eng_abstract,
               
               'rubric':data['root']['item']['rubrics']['rubric']['#text'],
               'rubric_code':data['root']['item']['rubrics']['rubric']['@code'],
               'journal_name':data['root']['item']['journal']['#text'],
               'journal_id':data['root']['item']['journal']['@id'],
               'journal_issn':data['root']['item']['journal']['@issn'],
               'journal_eissn':data['root']['item']['journal']['@eissn'],
               'journnal_impact_factor':data['root']['item']['journal']['@impactFactor'],
               'journal_VAK':data['root']['item']['journal']['@VAK'],
               'journal_Scopus':data['root']['item']['journal']['@Scopus'],
               'journal_WebofScience':data['root']['item']['journal']['@WebofScience'],
               'journal_RSCI':data['root']['item']['journal']['@RSCI'],
               'journal_RISC':data['root']['item']['journal']['@RISC'],
               'volume':volume,
               'journal_number':number,
               'journal_pages':jp,
               'number_of_pages':np,
               'publisher_id':publisher_id,
               'publisher':publisher,
               'DOI':code_DOI,
               'EDN':code_EDN,
               'UDK':code_UDK,
               'support': fin,
                               
               'genre_id':data['root']['item']['@genreId'],
               'cited':data['root']['item']['@cited'],
               'corecited':data['root']['item']['@coreCited'],
               'parent_id':data['root']['item']['@parentId'],
               'source_id':data['root']['item']['@sourceId'],
               'isFT':data['root']['item']['@isFT'],
               'isNEW':data['root']['item']['@isNew'],
               'link':data['root']['item']['link'],
               'url':url,
               'ref':data['root']['item']['ref']
              }
    
    # обновляем наш датафрейм df с помощью метода .concat() - присоединяем данные об одной статье;
    df = pd.concat([df, new_row], ignore_index=True)
    counter += 1 # обновляем счетчик
    
    # каждые 110 строк засыпаем на 1.8 секунды чтобы не перегрузить сервис
    if counter % 110 == 0:
        sleep(1.8)
    # каждые 1000 строк сохраняем промежуточную выгрузку в двух форматах - csv и excel
    if counter % 1000 == 0:
        df.to_csv(f'df_{current_date}_{counter}_rows.csv')
        df.to_excel(f'df_{current_date}_rows.xlsx')

100%|██████████| 18379/18379 [2:32:57<00:00,  2.00it/s]   


Здесь следует проверить, что размерность датафрейма `df` (`df.shape[0]`) совпадает с изначальным числом id статей, информацию по которым вы хотели выгрузить `len(lines)`.

In [ ]:
# если ответ True, то все хорошо.
df.shape[0] == len(lines)

In [92]:
# здесь следует проверить, что информация в pandas.DataFrame не дублируется.
df = df.drop_duplicates('id')
df

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,23064934,4,571249,1896,Фогель,Т.В.,Краснодарский государственный университет куль...,none,none,none,...,4,0,0,34056216,,0,1,http://elibrary.ru/item.asp?id=23064934,None,"Фогель, Т.В. Способы развития социально-помога..."
1,23049589,4,290282,1068,Воловская,Нина Михайловна,Новосибирский государственный университет экон...,Volovskaya,Nina Mikhailovna,Novosibirsk State University of Economics and ...,...,4,5,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049589,none,"Воловская, Н.М. СТУДЕНЧЕСКАЯ МОЛОДЕЖЬ И СОЦИАЛ..."
2,23049590,4,138991,436,Алибекова,Сиядат Яхьяевна,Региональный центр этнополитических исследован...,Alibekova,Siyadat Yakhyaevna,"Regional Centre of Ethnopolitical Researches, ...",...,4,1,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049590,none,"Алибекова, С.Я. ФОРМИРОВАНИЕ ЭТНОКОНФЕССИОНАЛЬ..."
3,23049640,4,674122,7608,Степанова,Наталья Юрьевна,Ижевский государственный технический университ...,Stepanova,Natalya Yuryevna,Izhevsk State Technical University,...,4,0,0,34055552,,1,0,http://elibrary.ru/item.asp?id=23049640,none,"Степанова, Н.Ю. НАЦИОНАЛЬНЫЙ КОМПОНЕНТ В ПРОГР..."
4,23051998,4,780229,,Бубнова,Ю.Г.,УПРАВЛЕНИЯ И АДМИНИСТРАТИВНО-ПРАВОВЫХ ДИСЦИПЛИ...,none,none,none,...,4,0,0,34055618,,1,1,http://elibrary.ru/item.asp?id=23051998,None,"Бубнова, Ю.Г. ВЛАСТЬ И ФОРМЫ ЕЕ ПРОЯВЛЕНИЯ НА ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66476,46499800,4,406338,1250,Передня,Дмитрий Григорьевич,Академия управления МВД России,Perednya,D.G.,Management Academy of the Ministry of the Inte...,...,4,0,0,46499795,,1,1,http://elibrary.ru/item.asp?id=46499800,None,"Передня, Д.Г. Интенсификация образовательного ..."
66477,46499861,4,510760,4812,Лескова,Ирина Валерьевна,ФГБОУ ВО «Российский государственный социальны...,Leskova,Irina Valerʹevna,Russian State Social University,...,4,0,0,46499841,,1,1,http://elibrary.ru/item.asp?id=46499861,https://sfk-mn.ru/PDF/24SCSK221.pdf,"Лескова, И.В. Мигранты армяне в российском соц..."
66478,46503369,4,622383,11,Долгов,Александр Юрьевич,none,Dolgov,A.Yu.,none,...,4,0,0,46503368,,1,1,http://elibrary.ru/item.asp?id=46503369,None,"Долгов, А.Ю. Социология смерти: продолжая труд..."
66479,46505702,4,491974,583,Бреславский,Анатолий Сергеевич,"Институт монголоведения, буддологии и тибетоло...",Breslavsky,Anatoly S.,"Institute for Mongolian, Buddhist and Tibetan ...",...,4,0,0,46505698,,1,1,http://elibrary.ru/item.asp?id=46505702,None,"Бреславский, А.С. Территориальное общественное..."


<h2><center>Шаг 3. Сохранение файлов</center></h2>

<b>Финальный результат - файлы <code>final_data.csv</code>, <code>final_data.xlsx</code></b>

In [93]:
df.to_csv('final_data.csv')
df.to_excel('final_data.xlsx')